<a href="https://colab.research.google.com/github/ashraful009/Banglish-Abuse-Words-detection/blob/main/Banglish_Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!pip install -q transformers datasets torch scikit-learn imbalanced-learn xgboost gradio streamlit matplotlib seaborn sentencepiece tokenizers

# STEP 1 - Imports & Utility functions

In [5]:
import os
import random
import json
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN

print('GPU available:', torch.cuda.is_available())
print('Torch device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')

# Helpers
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

OUTPUT_DIR = Path('colab_results')
OUTPUT_DIR.mkdir(exist_ok=True)

GPU available: True
Torch device: Tesla T4


# Load Datasets

In [6]:
file_path = 'Banglish_Hate_Speech_Dataset.xlsx'
if not os.path.exists(file_path):
    print('Warning: dataset file not found at', file_path)
else:
    df = pd.read_excel(file_path)
    print('Loaded:', df.shape)
    display(df.head())

# Quick check for common column names
if 'text' not in df.columns or 'label' not in df.columns:
    print('Expected columns `text` and `label`. Found:', df.columns.tolist())

Loaded: (5000, 4)


,Unnamed: 0,Comment,Hate,Type
0,NaN,Kanki ki der Allah sob samoy valo rake,Yes,Others
1,NaN,khanki akta,Yes,Sexual
2,NaN,Ore saniliyon,Yes,Sexual
3,NaN,Bokachoda Shangbadik,Yes,Slang
4,NaN,Balda dekhaiche,Yes,Slang


Expected columns `text` and `label`. Found: ['Unnamed: 0', 'Comment', 'Hate', 'Type']
